In [292]:
import numpy as np
import matplotlib.pyplot as plt
import random
from itertools import product
import pandas as pd

In [293]:
p_quebrar = lambda x: pq0/(pq0+(1-pq0)*np.exp(-alfa*x))

custos_acao0 = [[0,-100],[1/9,-10],[2/9,10],[1/3,20],[2/9,25],[1/9,30]] #nao fazer nada
custos_acao1 = [[1,-500]] #substituicao
custos_acao2 = [[1,-10]] # manutencao

def valor_acao(estado,idx_acao,prob_q):
    if idx_acao == 0:
        pq = prob_q(estado[1])
        valor = pq*custos_acao0[0][1] + (1-pq)*sum([x[0]*x[1] for x in custos_acao0[1:]])
    elif idx_acao == 1:
        valor = sum([x[0]*x[1] for x in custos_acao1])
    elif idx_acao == 2:
        valor = sum([x[0]*x[1] for x in custos_acao2])
    
    return valor

def reward_estado(estado,probs,idx_ac):
    rw = 0
    for i in idx_ac:
        rw = rw +  probs[i]*valor_acao(estado,i,p_quebrar)
    
    return rw

In [320]:
def constroe_estados(tx,ty):
    estados = {}
    estados["0"] = [(0,0)]
    estados_list = []
    for v in range(tx):
        aux = list(product([v+1],[-1]+list(range(0,(v+1)*(ty-1)+1))))
        estados[str(v+1)] =  aux
        estados_list = estados_list + aux  
    return estados,estados_list

def proximos(estado,ty):
    atingiveis = {}
        
    if estado[1]>=0:
        aux_0 = list(set(list(product([estado[0]+1],[-1]+[estado[1]+i for i in range(ty)]))))
        ac_0 = [0]*len(aux_0)
        
        aux_1 = [(estado[0]+1,0)]
    
        aux_2 = [(0,0)]
    
        atingiveis["estados"] = aux_0 + aux_1+ aux_2
        probs = []
        for y in aux_0:
            probs.append(transicao(estado,y,0,ty))
        
        probs = probs + [1,1]
        ac = ac_0+[1,2]


        atingiveis["acoes"] = ac
        atingiveis["probs"] = probs
    
    return atingiveis

def define_probs(ty):
    aux = list(range(0,ty))
    # cr1 = [(x+1)/ty for x in aux]
    # cr2 = [1-x/ty for x in aux]
    # cr = np.array([min(x,y) for x,y in zip(cr1,cr2)])

    mu =  np.mean(aux)
    s2i = 1/np.var(aux)
    cr = [np.exp(-s2i*(x-mu)**2) for x in aux]
    return cr/sum(cr) 

def transicao(estado1,estado2,acao,ty):
    if acao == 0:
        aux_q_km = pq0/(pq0+(1-pq0)*np.exp(-alfa*estado1[1]))
        # aux_q_t = pq0/(pq0+(1-pq0)*np.exp(-alfa*estado1[1]))

        pq = aux_q_km   
        probs = (1-pq)*define_probs(ty)
        dx0 = estado2[0]-estado1[0]
        if dx0==1 and estado1[1]>=0:
            dy0 = estado2[1]-estado1[1]
            if dy0 in list(range(ty)) and estado2[1]>=0:
                pr = probs[dy0]
            elif estado2[1]==-1:
                pr = pq 
            else:
                pr = 0  
        else:
            pr = 0
            
    elif acao == 1:
        dx0 = estado2[0]-estado1[0]
        if dx0==1 and estado1[0]==0:
            pr = 1
        else:
            pr = 0
        
    elif acao == 2:
        if estado2[1] == 0 and estado2[0] == 0:
            pr = 1
        else:
            pr = 0
    else:
        pr = 0

    return pr



pq0, alfa = 0.01 , 0.5


In [ ]:
tx,ty = 5,3
_ , est_list = constroe_estados(tx,ty)
estado1 = []
estado2 = []
acao = []
probs = []
for x in est_list:
    for y in est_list:
        estado1.append(x)
        estado2.append(y)
        acao.append(0)
        probs.append(transicao(x,y,0,ty))

df = pd.DataFrame({'estado origem':estado1,'estado_chegada':estado2,'acao':acao,"probs":probs})
df

In [ ]:
df[df.probs>0]

In [326]:
transicao((3,-1),(4,0),0,5)

0

2.4837318858984925

In [330]:
pd.DataFrame(proximos((4,0),5))

,estados,acoes,probs
0,"(5, -1)",0,0.010000
1,"(5, 4)",0,0.053944
2,"(5, 1)",0,0.241759
3,"(5, 0)",0,0.053944
4,"(5, 3)",0,0.241759
5,"(5, 2)",0,0.398594
6,"(5, 0)",1,1.000000
7,"(0, 0)",2,1.000000
